In [1]:
import numpy as np
import os
import time
from math import sqrt
from pathfinding.core.diagonal_movement import DiagonalMovement
from pathfinding.core.grid import Grid
from pathfinding.finder.a_star import AStarFinder

np.set_printoptions(threshold=np.inf)

In [2]:
def input(fname):
    fname = os.path.join(fname)

    data = []
    with open(fname) as f:
        for line in f:
            data.append(line.strip())
    return data  

In [25]:
# risk_map = input('input.txt')
risk_map = input('test-input.txt')
risk_map = np.array([[int(i) for i in d] for d in risk_map])
print(len(risk_map))
risk_map

10


array([[1, 1, 6, 3, 7, 5, 1, 7, 4, 2],
       [1, 3, 8, 1, 3, 7, 3, 6, 7, 2],
       [2, 1, 3, 6, 5, 1, 1, 3, 2, 8],
       [3, 6, 9, 4, 9, 3, 1, 5, 6, 9],
       [7, 4, 6, 3, 4, 1, 7, 1, 1, 1],
       [1, 3, 1, 9, 1, 2, 8, 1, 3, 7],
       [1, 3, 5, 9, 9, 1, 2, 4, 2, 1],
       [3, 1, 2, 5, 4, 2, 1, 6, 3, 9],
       [1, 2, 9, 3, 1, 3, 8, 5, 2, 1],
       [2, 3, 1, 1, 9, 4, 4, 5, 8, 1]])

In [44]:
class Node:
    def __init__(self, parent=None, x=None, val=None):
        self.parent = parent
        self.x = x
        self.val = val

        self.g = 0
        self.h = 0
        self.f = 0

class AStarPathFinder:
    """square grid only right now"""
    def __init__(self,grid,start,end,diag=False):
        self.grid = grid
        self.diag = diag

        self.start = Node(None, start, grid[start])
        self.end = Node(None, end, grid[start])
        self.path_val = 0

        self.x_min = 0
        self.x_max = len(grid)

    def cost(self,x):
        if self.diag:
            dist = lambda x1,x2: sqrt((x1[0]-x2[0])**2 + abs(x[1]-x2[1])**2)
        else:
            dist = lambda x1,x2: abs(x1[0]-x2[0]) + abs(x1[1]-x2[1])
        g = dist(x,self.start)
        h = dist(x,self.end)
        return g + h, g, h

    def neighbors(self, x):
        x_nn = []
        x_nn.append([x[0]-1, x[1]  ])
        x_nn.append([x[0]  , x[1]+1])
        x_nn.append([x[0]+1, x[1]  ])
        x_nn.append([x[0]  , x[1]-1])
        if self.diag:
            x_nn.append([x[0]+1, x[1]+1])
            x_nn.append([x[0]-1, x[1]+1])
            x_nn.append([x[0]+1, x[1]-1])
            x_nn.append([x[0]-1, x[1]-1])
        x_nn = np.array(x_nn)

        # SQUARE ONLY PART HERE
        x_nn = np.delete(x_nn,np.where(x_nn<x_min)[0], axis=0)
        x_nn = np.delete(x_nn,np.where(x_nn>x_max)[0], axis=0)

        return (tuple(l) for l in x_nn) 

    def find_path(self, start, end):
            open_list = [start]
            closed_list = []

            _, current = min([(n.val, n) for n in open_list])
            open_list.remove(current)
            closed_list.append(current)

            nn_list = self.neighbors(current)
            for nn in nn_list:
                if nn not in closed_list:
                    if nn not in open_list:
                        open_list.append(nn)
                    



In [46]:
start = (0,0)
grid = risk_map
open_list = [ (1,0), (0,0), (1,1), (2,2)]
x, current = min([(x, grid[x]) for x in open_list])
open_list.remove(x)
[x for [1,2,3] not in [1,2]]


(0, 0)
1


In [14]:
grid = Grid(matrix=risk_map)

start = grid.node(0,0)
# end = grid.node(risk_map.shape[0]-1,risk_map.shape[1]-1)
end = grid.node(len(risk_map)-1,len(risk_map)-1)

pfinder = AStarFinder(diagonal_movement=DiagonalMovement.never)
path, runs = pfinder.find_path(start, end, grid)

print('operations:', runs, 'path length:', len(path))
# print(grid.grid_str(path=path, start=start, end=end))

sum = 0
for p in path:
  sum += risk_map[p[1]][p[0]]
  # print(p, risk_map[p[1],p[0]])
print(sum - risk_map[0][0])


operations: 86 path length: 19
40


In [5]:
r = 5
big_map = np.zeros((risk_map.shape[0]*r,risk_map.shape[1]*r))

n = risk_map.shape[0]

big_map[0:n,0:n] = risk_map
for i in range(1,r):
    big_map[0:n,i*n:(i+1)*n] = big_map[0:n,(i-1)*n:i*n]+1
    big_map[np.where(big_map>9)] = 1
for i in range(1,r):
    big_map[i*n:(i+1)*n,0:n*r] = big_map[(i-1)*n:i*n,0:n*r]+1
    big_map[np.where(big_map>9)] = 1

grid = Grid(matrix=big_map)

start = grid.node(0,0)
end = grid.node(big_map.shape[0]-1,big_map.shape[1]-1)

finder = BiAStarFinder(diagonal_movement=DiagonalMovement.never)
path, runs = finder.find_path(start, end, grid)

print('operations:', runs, 'path length:', len(path))
# print(grid.grid_str(path=path, start=start, end=end))

sum = 0
for p in path:
  sum += big_map[p[1],p[0]]
  # print(p, risk_map[p[1],p[0]])
print(sum - big_map[0,0])

NameError: name 'BiAStarFinder' is not defined